<a href="https://colab.research.google.com/github/TonyLiu836/NBA_MVP_Predictor/blob/main/NBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nba_api
!pip install requests

In [2]:
import pandas as pd

### Data Collection
    

In [54]:
from nba_api.stats.endpoints import teamplayerdashboard
from nba_api.stats.endpoints import leaguedashteamstats
from nba_api.stats.endpoints import playerdashboardbyyearoveryear

# Get list of teams that played in a season
leagueteams = leaguedashteamstats.LeagueDashTeamStats(season='2020-21')
teams = leagueteams.get_data_frames()[0]
team_list = teams.loc[:, ['TEAM_ID','TEAM_NAME']]
print(team_list.to_markdown(), '\n')

def get_teams(start_year):
    end_year = (start_year + 1) % 100
    leagueteams = leaguedashteamstats.LeagueDashTeamStats(season=f'{start_year}-{end_year:02}')
    teams = leagueteams.get_data_frames()[0]
    return teams.loc[:, 'TEAM_ID']


# Get list of players that played in a season
bulls_id = teams.at[4, 'TEAM_ID']
teamplayers = teamplayerdashboard.TeamPlayerDashboard(bulls_id, season='2020-21')
players = teamplayers.get_data_frames()[1]
good_players = players[players.loc[:,'MIN'] > 800]
players_list = good_players.loc[:, ['PLAYER_ID', 'PLAYER_NAME']]
print(players_list.to_markdown(), '\n')

def get_players(team_id, start_year):
    end_year = (start_year + 1) % 100
    teamplayers = teamplayerdashboard.TeamPlayerDashboard(team_id, season=f'{start_year}-{end_year:02}')
    players = teamplayers.get_data_frames()[1]
    good_players = players[players.loc[:,'MIN'] > 800]
    return good_players.loc[:, ['PLAYER_ID', 'PLAYER_NAME']]

# Get player stats for a season
vuc_id = good_players.at[13, 'PLAYER_ID']
playerdashboard = playerdashboardbyyearoveryear.PlayerDashboardByYearOverYear(vuc_id, per_mode_detailed='PerGame')
stats = playerdashboard.get_data_frames()[1]
seasonstats = stats[stats['GROUP_VALUE'] == '2020-21']
print(seasonstats.columns)
print(seasonstats.loc[:, ['GROUP_VALUE', 'TEAM_ABBREVIATION', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'REB', 'AST', 'TOV', 'STL', 'BLK', 'PTS',
       'PLUS_MINUS']])

def get_player_stats(player_id, start_year):
    end_year = (start_year + 1) % 100
    playerdashboard = playerdashboardbyyearoveryear.PlayerDashboardByYearOverYear(player_id, per_mode_detailed='PerGame')
    stats = playerdashboard.get_data_frames()[1]
    seasonstats = stats[stats['GROUP_VALUE'] == f'{start_year}-{end_year:02}']
    seasonstats.drop(columns=['GROUP_SET', 'MAX_GAME_DATE', 'CFID', 'CFPARAMS'])
    return seasonstats

|    |    TEAM_ID | TEAM_NAME              |
|---:|-----------:|:-----------------------|
|  0 | 1610612737 | Atlanta Hawks          |
|  1 | 1610612738 | Boston Celtics         |
|  2 | 1610612751 | Brooklyn Nets          |
|  3 | 1610612766 | Charlotte Hornets      |
|  4 | 1610612741 | Chicago Bulls          |
|  5 | 1610612739 | Cleveland Cavaliers    |
|  6 | 1610612742 | Dallas Mavericks       |
|  7 | 1610612743 | Denver Nuggets         |
|  8 | 1610612765 | Detroit Pistons        |
|  9 | 1610612744 | Golden State Warriors  |
| 10 | 1610612745 | Houston Rockets        |
| 11 | 1610612754 | Indiana Pacers         |
| 12 | 1610612746 | LA Clippers            |
| 13 | 1610612747 | Los Angeles Lakers     |
| 14 | 1610612763 | Memphis Grizzlies      |
| 15 | 1610612748 | Miami Heat             |
| 16 | 1610612749 | Milwaukee Bucks        |
| 17 | 1610612750 | Minnesota Timberwolves |
| 18 | 1610612740 | New Orleans Pelicans   |
| 19 | 1610612752 | New York Knicks        |
| 20 | 161

In [ ]:
import pandas as pd
from nba_api.stats.endpoints import teamplayerdashboard
from nba_api.stats.endpoints import leaguedashteamstats

data = []
for start_year in range(1996, 2022):
    print(start_year)
    leagueteams = get_teams(start_year)
    for team_id in leagueteams:
        players = get_players(team_id, start_year)
        for player in players.itertuples():
            print(player)
            stats = get_player_stats(player[1], start_year)
            print(stats)
            stats.insert(0, "PLAYER_NAME", player[2])
            data.append(stats)
            
    

1996
Pandas(Index=2, PLAYER_ID=363, PLAYER_NAME='Christian Laettner')
   GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
10   By Year     1996-97  1610612737               ATL  1997-04-20T00:00:00   

    GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
10  82  56  26  0.683  38.2  ...          1        2         5         1   

    PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
10                1                     1         1         1   264   1996-97  

[1 rows x 65 columns]
Pandas(Index=5, PLAYER_ID=87, PLAYER_NAME='Dikembe Mutombo')
   GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
14   By Year     1996-97  1610612737               ATL  1997-04-20T00:00:00   

    GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
14  80  54  26  0.675  37.2  ...          1        1        10         2   

    PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK 

  GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
3   By Year     1996-97  1610612738               BOS  1997-04-20T00:00:00   

   GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
3  81  14  67  0.173  28.2  ...          1        1         2         1   

   PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
3                4                     1         1         1   264   1996-97  

[1 rows x 65 columns]
Pandas(Index=1, PLAYER_ID=193, PLAYER_NAME='Anthony Mason')
  GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
5   By Year     1996-97  1610612766               CHH  1997-04-20T00:00:00   

   GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
5  73  48  25  0.658  43.0  ...          2        2         3         1   

   PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
5                3                     1         1         1   2

  GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
7   By Year     1996-97  1610612741               CHI  1997-04-19T00:00:00   

   GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
7  82  69  13  0.841  37.7  ...          3        3         6         1   

   PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
7                1                     1         1         2   264   1996-97  

[1 rows x 65 columns]
Pandas(Index=13, PLAYER_ID=70, PLAYER_NAME='Steve Kerr')
  GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
6   By Year     1996-97  1610612741               CHI  1997-04-19T00:00:00   

   GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
6  82  69  13  0.841  22.7  ...          2        2         2         1   

   PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
6                1                     1         1         1   264 

  GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
2   By Year     1996-97  1610612742               DAL  1997-04-19T00:00:00   

   GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
2  75  23  52  0.307  29.5  ...          1        2         2         1   

   PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
2                3                     1         1         1   264   1996-97  

[1 rows x 65 columns]
Pandas(Index=4, PLAYER_ID=376, PLAYER_NAME='Eric Montross')
   GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
9    By Year     1996-97          -1               TOT  1997-04-20T00:00:00   
10   By Year     1996-97  1610612751               NJN  1997-04-20T00:00:00   
11   By Year     1996-97  1610612742               DAL  1997-02-15T00:00:00   

    GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
9   78  27  51  0.346  23.4  ...          1        1      

  GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
3   By Year     1996-97          -1               TOT  1997-04-19T00:00:00   
4   By Year     1996-97  1610612743               DEN  1997-04-19T00:00:00   
5   By Year     1996-97  1610612762               UTA  1996-11-13T00:00:00   

   GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
3  67  18  49  0.269  15.8  ...          1        1         1         1   
4  65  16  49  0.246  16.1  ...          1        1         1         1   
5   2   2   0  1.000   4.1  ...          4        3         1         4   

   PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
3                2                     1         1         1   264   1996-97  
4                3                     1         1         1   264   1996-97  
5                4                     4         2         1   264   1996-97  

[3 rows x 65 columns]
Pandas(Index=4, PLAYER_ID=179, PLAYER_NAME='Bry

...

  GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
7   By Year     1996-97  1610612750               MIN  1997-04-19T00:00:00   

   GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
7  68  30  38  0.441  24.5  ...          2        2         2         1   

   PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
7                2                     2         2         1   264   1996-97  

[1 rows x 65 columns]
Pandas(Index=4, PLAYER_ID=28, PLAYER_NAME='Doug West')
  GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
4   By Year     1996-97  1610612750               MIN  1997-04-19T00:00:00   

   GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
4  68  31  37  0.456  28.3  ...          1        1         3         1   

   PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
4                3                     1         1         1   264   

  GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
7   By Year     1996-97          -1               TOT  1997-04-19T00:00:00   
8   By Year     1996-97  1610612742               DAL  1997-04-19T00:00:00   
9   By Year     1996-97  1610612751               NJN  1997-01-20T00:00:00   

   GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
7  54  12  42  0.222  33.0  ...          1        1         1         1   
8  20   2  18  0.100  29.9  ...          3        1         3         2   
9  34  10  24  0.294  34.9  ...          1        2         3         1   

   PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
7                1                     1         1         1   264   1996-97  
8                7                     2         5         2   264   1996-97  
9                8                     1         1         2   264   1996-97  

[3 rows x 65 columns]
Pandas(Index=18, PLAYER_ID=762, PLAYER_NAME='Sh

   GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
13   By Year     1996-97  1610612753               ORL  1997-04-19T00:00:00   

    GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
13  67  34  33  0.507  15.1  ...          6        8         6         9   

    PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
13                7                     9         6         1   264   1996-97  

[1 rows x 65 columns]
Pandas(Index=8, PLAYER_ID=192, PLAYER_NAME='Dennis Scott')
  GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
7   By Year     1996-97  1610612753               ORL  1997-04-19T00:00:00   

   GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
7  66  38  28  0.576  32.8  ...          2        2         2         2   

   PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
7                3                     2         1         

   GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
12   By Year     1996-97  1610612755               PHI  1997-04-05T00:00:00   

    GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
12  62  18  44   0.29  21.3  ...          2        2         5         4   

    PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
12                9                     2         2         1   264   1996-97  

[1 rows x 65 columns]
Pandas(Index=2, PLAYER_ID=76, PLAYER_NAME='Cedric Ceballos')
   GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
8    By Year     1996-97          -1               TOT  1997-04-19T00:00:00   
9    By Year     1996-97  1610612756               PHX  1997-04-19T00:00:00   
10   By Year     1996-97  1610612747               LAL  1996-11-13T00:00:00   

    GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
8   50  29  21  0.580  28.6  ...          1        

   GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
14   By Year     1996-97  1610612757               POR  1997-04-20T00:00:00   

    GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
14  82  49  33  0.598  37.6  ...          1        3         3         1   

    PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
14                1                     1         1         1   264   1996-97  

[1 rows x 65 columns]
Pandas(Index=14, PLAYER_ID=739, PLAYER_NAME='Rasheed Wallace')
   GROUP_SET GROUP_VALUE     TEAM_ID TEAM_ABBREVIATION        MAX_GAME_DATE  \
17   By Year     1996-97  1610612757               POR  1997-04-20T00:00:00   

    GP   W   L  W_PCT   MIN  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
17  62  37  25  0.597  30.6  ...          6        3         8         7   

    PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  CFID  CFPARAMS  
17                6                    14         

In [52]:
print(len(data))
big_data = pd.concat(data)

,PLAYER_NAME,GROUP_SET,GROUP_VALUE,TEAM_ID,TEAM_ABBREVIATION,MAX_GAME_DATE,GP,W,L,W_PCT,...,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,CFID,CFPARAMS
8,Anthony Miller,By Year,1996-97,1610612737,ATL,1996-11-16T00:00:00,1,0,1,0.0,...,1,1,3,6,8,2,2,1,264,1996-97


In [36]:
from nba_api.stats.static import players
mj = players.find_players_by_full_name('Michael Jordan')
mj_id = mj[0]['id']
#get_player_stats(202696, 2018)
players.find_player_by_id(202696)

{'id': 202696,
 'full_name': 'Nikola Vucevic',
 'first_name': 'Nikola',
 'last_name': 'Vucevic',
 'is_active': True}